## Protocole à suivre pour l'ensemble de la procédure

### Importation des librairies

In [ ]:
import pandas
pandas.__version__
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


### Création du fichier avec les colonnes date, numero compta et plan, débit, crédit, solde progressif

#### Note

##### il peut arriver que read_excel ne fonctionne pas soit le chemin n'est pas bon soit le fichier qu'on veut ouvrir est trop lourd 
##### si le fichier est trop lourd, Fichier -> Enregistrer sous... -> Format du fichier -> classeur excel binaire
##### exécution à faire une fois : Terminal -> pip install pyxlsb

#### Code

In [ ]:
# le read_excel prend en argument le chemin qui va le mener vers le fichier voulu
grand_livre = pd.read_excel("Grand livre 2020-2021.xls")


In [ ]:
# pour vérifier qu'on a bien le fichier, afficher le dataframe 
grand_livre


In [ ]:
# on sélectionne les colonnes qu'on veut garder, elles s'afficheront dans l'order d'écriture
data = grand_livre[["MON BRICO ETS AH-HOT","Unnamed: 1","Unnamed: 14","Unnamed: 16","Unnamed: 18"]]
# on sélectionne les lignes qui nous intéressent, ie. celles qui ont un solde ou un numéro compta ou une date
data_filtered_2 = data[data["Unnamed: 18"].notna() | data["Unnamed: 1"].notna()]
data_filtered_3 = data_filtered_2[data_filtered_2["MON BRICO ETS AH-HOT"] != "Date"]
data_filtered_4 = data_filtered_3[data_filtered_3["MON BRICO ETS AH-HOT"] != "Total Grand Livre"]

In [ ]:
# vérification du dataframe
data_filtered_4


In [ ]:
# il est possible que les deux étapes qui suivent affichent une erreur
# le code devrait fonctionner malgré tout 
data_filtered_5 = data_filtered_4


In [ ]:
# ce bloc permet d'attribuer à chaque date et solde leur numéro compta et de filtrer les lignes
for i in data_filtered_5["Unnamed: 1"].index: 
    if pd.isna(data_filtered_4["Unnamed: 1"][i]) :
        data_filtered_5["Unnamed: 1"][i] = data_filtered_5["Unnamed: 1"][i-1]
        
data_filtered_6 = data_filtered_5[data_filtered_5["Unnamed: 18"].notna()]
data_filtered_7 = data_filtered_6[data_filtered_6["MON BRICO ETS AH-HOT"].notna()]



In [ ]:
# vérification du dataframe
data_filtered_7


### Renommer les colonnes


In [ ]:
# attention, python est sensible à la casse
data_filtered_7 = data_filtered_7.rename(columns = {'MON BRICO ETS AH-HOT' : 'date', 'Unnamed: 1': 'plan_compta', 'Unnamed: 14': 'debit', 'Unnamed: 16':'credit', 'Unnamed: 18': 'solde progressif'})

In [ ]:
# vérification
data_filtered_7

### Split des plans comptables

#### permet de séparer les numéro des intitulés comptables

In [ ]:
# supprimer le mot 'Suite' 
data_filtered_7['plan_compta'] = data_filtered_7['plan_compta'].str.replace('Suite ','')


In [ ]:
# supprimer les chaines de caractères après le premier " " pour ne garder que le numéro compta
var_test = data_filtered_7['plan_compta'].str.split(' ').str[0]


In [ ]:
# vérifier qu'il ne renvoie que les numéros, sans la description
var_test


In [ ]:
data_filtered_7['plan_compta'] = var_test


In [ ]:
# vérification
data_filtered_7


### Merge de deux dataframes

In [ ]:
# on va merge avec le plan comptable 
# si le plan comptable est dans un dossier, faire attention au chemin 
plan_comptable = pd.read_excel("TR__Plan_comptable/Plan comptable.xls")



In [ ]:
# renommer la colonne 'N° du compte' pour pouvoir faire le lien des deux dataframes
plan_comptable = plan_comptable.rename(columns = {'nom_colonne' : 'plan_compta'})

In [ ]:
#afficher le plan compable
plan_comptable


In [ ]:
grand_livre_V1 = data_filtered_7.merge(plan_comptable, how='left', on='plan_compta')


In [ ]:
# vérification 
grand_livre_V1


In [ ]:
# sélection des colonnes qui nous intéressent
grand_livre_V2 = grand_livre_V1[["date","plan_compta","Intitulé du compte général","debit","credit","solde progressif"]]

In [ ]:
# vérification 
grand_livre_V2


### Création d'une colonne avec le premier chiffre du numéro compta

In [ ]:
# il est possible que les deux blocs qui suivent affichent une erreur
grand_livre_V2["categorie"] = grand_livre_V2["plan_compta"]


In [ ]:
# une erreur mais ça fonctionne
for i in range (grand_livre_V2.shape[0]):
    grand_livre_V2["categorie"][i] = grand_livre_V2["plan_compta"][i][0:1]
    
    

In [ ]:
# vérification
grand_livre_V2

### Création de la colonne "solde" , dérivée des colonnes existantes

In [ ]:
# on remplace les cellules vides de débit et de crédit par des 0
# affichage d'erreurs possible, ne pas en tenir compte
grand_livre_V2['debit'] = grand_livre_V2["debit"].fillna(0)

In [ ]:
grand_livre_V2["debit"]

In [ ]:
grand_livre_V2['credit'] = grand_livre_V2["credit"].fillna(0)

In [ ]:
grand_livre_V2["credit"]

In [ ]:
grand_livre_V2["solde"] = grand_livre_V2["debit"] -  grand_livre_V2["credit"]

In [ ]:
grand_livre_V2

### Mise en page

#### Création des colonnes mois / annee

In [ ]:
# mise en forme de la date en format date
grand_livre_V2["date"] = pd.to_datetime(grand_livre_V2["date"], format='%d/%m/%y')


In [ ]:
# vérification 
grand_livre_V2


In [ ]:
# des erreurs pourront apparaitre
grand_livre_V2["mois"] = grand_livre_V2["date"]


In [ ]:
grand_livre_V2["mois"] = grand_livre_V2["date"].dt.month


In [ ]:
grand_livre_V2["annee"] = grand_livre_V2["date"]


In [ ]:
grand_livre_V2["annee"] = grand_livre_V2["date"].dt.year


#### Modifier l'ordre des colonnes

In [ ]:
grand_livre_V2 = grand_livre_V2[['annee','mois','date','categorie','plan_compta','Intitulé du compte général','debit','credit','solde','solde progressif']]


In [ ]:
# vérification
grand_livre_V2


### Création d'un nouveau fichier excel 


In [ ]:
# création du fichier
# indiquer entre guillemets le nom du fichier qu'on veut créer
file_name = 'QR_rendu_2020-2021.xlsx'


In [ ]:
grand_livre_V2.to_excel(file_name)
print('Sales record successfully exported into Excel File')
# un nouveau fichier excel a été créé
# sur le excel, supprimer la premiere colonne


#### Note 

In [ ]:
# chaque fois que vous exécutez les blocs "création d'un nouveau fichier excel", si le fichier existe déjà,
# il va être modifié 

### Concaténation de deux dataframes

In [ ]:
# le code écrit précedemment permet de trier un fichier excel, par défaut un nom de fichier est indiqué
# exécuter le code pour tous les fichiers nécessaires en modifiant le code



In [ ]:
rendu_2019_2020 = pd.read_excel("QR_rendu_2019-2020.xlsx")



In [ ]:
rendu_2020_2021 = pd.read_excel("QR_rendu_2020-2021.xlsx")



In [ ]:
# the names of columns of both files have to be the same
rendu_2019_2021 = pd.concat([rendu_2019_2020, rendu_2020_2021], axis=0)
# une fois terminé, reprendre "création d'un nouveau fichier excel, en modifiant les paramètres
# file_name = 'le_nom_du_fichier'
# le_nom_du_fichier.to_excel(file_name)


In [ ]:
# vérification
rendu_2019_2021

In [ ]:
# supprimer la colonne unnamed 0
rendu_2019_2021 = rendu_2019_2021.drop(columns=["Unnamed: 0"])

In [ ]:
rendu_2019_2021

#### Rendu

In [ ]:
# pour enregistrer l'excel, exécuter les blocs "création d'un nouveau fichier excel"
# création du fichier
# indiquer entre guillemets le nom du fichier qu'on veut créer
file_name = 'QR_rendu_2019-2021.xlsx'
rendu_2019_2021.to_excel(file_name)
print('Sales record successfully exported into Excel File')
# un nouveau fichier excel a été créé
# sur le excel, supprimer la premiere colonne

### Outils qui pourraient servir

#### exemple de recherche par mois et année

In [ ]:
mars_2020 = rendu_2019_2021.loc[(rendu_2019_2021["date"] >= "2020-03-01") & (rendu_2019_2021["date"] <= "2020-03-31")]


#### tracer un graphe

In [ ]:
nom_fichier = pd.read_excel("nom_fichier.xlsx")


In [ ]:
nom_fichier


In [ ]:
plt_av = nom_fichier[nom_fichier["nom_colonne"] == "attribut_colonne"]

In [ ]:
# affichage
plt_av.plot.scatter(x="dates", y="solde progressif", alpha=0.5)

### Dernière manipulation sur excel

In [ ]:
# supprimer la première colonne
# date en type Date courte
# c'est fini ! 